In [1]:
#https://nlml.github.io/in-raw-numpy/in-raw-numpy-t-sne/

import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from tqdm import tqdm

def keep_numbers(numbers: list[int, ...], dataset) -> torch.Tensor:
    a = np.array([(dataset.targets == i).numpy() for i in numbers])
    keeps = [False for i in range(len(a[0]))]
    for data_point in range(len(a[0])):
        for number in range(len(numbers)):
            if a[:, data_point][number]:
                keeps[data_point] = True
    return torch.tensor(keeps, dtype=None)


# N.B. don't apply one-hot encoding on labels - we don't need them (unsupervised learning)
# Applying ToTensor() transform to change type(data) to torch.Tensor
my_transform_1 = torchvision.transforms.ToTensor()
my_transforms = torchvision.transforms.Compose([my_transform_1])

# Set download=True first time to acquire dataset
_MNIST_DATASET = torchvision.datasets.MNIST(root="data/",
                                            download=False,
                                            train=True,
                                            transform=my_transforms)

# Picking out numbers from dataset
_numbers = [0,1,2,3,4,5,6,7,8,9]
_MNIST_DATASET.data = _MNIST_DATASET.data[keep_numbers(_numbers, _MNIST_DATASET)]
_MNIST_DATASET.targets = _MNIST_DATASET.targets[keep_numbers(_numbers, _MNIST_DATASET)]


In [2]:
_MNIST_DATASET.__len__()

60000

In [10]:
# Normalizing w. respect to max pixel val (255)
data_points = []
for data_point in tqdm(range(_MNIST_DATASET.__len__())):
    data_points.append(_MNIST_DATASET.data[data_point].flatten().to(torch.float) / 255.0)
# Normalizing with respect to max label val
labels = []
for label in tqdm(range(len(_MNIST_DATASET.targets))):
    labels.append(_MNIST_DATASET.targets[label].to(torch.float) / 9.0)

100%|██████████| 60000/60000 [00:00<00:00, 128297.83it/s]


In [5]:
# Calculating P_j|i
P_ij_values = []
for j in range(len(data_points)):
    x_j = data_points[j]
    for i in range(len(data_points)):
        if i != j:
            x_i = data_points[i]
            _numerator = torch.exp(torch.pow(-torch.norm(x_i-x_j),2))
            _denominator = 0
            for k in range(len(data_points)):
                if i != k:
                    x_k = data_points[k]
                    _denominator += torch.exp(torch.pow(-torch.norm(x_i-x_k),2))

torch.Size([784])

In [7]:
a, b = torch.tensor([1,2,3]), torch.tensor([1,1,1])
torch.pow(a-b,2)

tensor([0, 1, 4])